# Matchbox Educable Noughts and Crosses Engine



Each box corresponds to a gamestate, and has a list of moves it can make and corresponding bead counts.

In [71]:
from dataclasses import dataclass


@dataclass
class Box:
    state: str      # state should be generic but dataclass does not seem to support generic types????????????????
    moves: list[str]
    beads: list[int]
    



The gamestate in naughts and crosses can be represented as a string of nine characters.
for example the string `XX-OX--OO"` represents the gamestate:

|X|X||
|---|---|---|
|O|X||
||O|O|

The entire MENACE machine can be thought of as a mapping from states to boxes:

In [72]:
# type MENACE = dict[str, Box[str]]

In order to reduce the statespace we should ignore treat states that are symmetrical or rotationally symmetrical as redundant.
To do this we create functions for flipping and rotating the board:

In [73]:
def flipX(board:str):
    "flips the board over the X axis"
    return board[6:9] + board[3:6] + board[:3]

def rotate(board:str):
    b = []
    b.append(board[6])
    b.append(board[3])
    b.append(board[0])
    b.append(board[7])
    b.append(board[4])
    b.append(board[1])
    b.append(board[8])
    b.append(board[5])
    b.append(board[2])
    
    return "".join(b)

def all_rotations(board: str):
    rs = [board]
    for _ in range(3):
        board = rotate(board)
        rs.append(board)
    return rs

def all_versions(board:str):
    return all_rotations(board) + all_rotations(flipX(board))

In [102]:
from itertools import permutations
print([p for p in permutations("som")])

[('s', 'o', 'm'), ('s', 'm', 'o'), ('o', 's', 'm'), ('o', 'm', 's'), ('m', 's', 'o'), ('m', 'o', 's')]


In [138]:
from itertools import permutations

boards = list({"".join(p)  for b in ["X"* i + "O" * j + "-" * ((9 - i - j)) for i in range(5) for j in (i, i + 1)] for p in permutations(b)})

symmetric_board = {}
for b in boards:
    if b not in symmetric_board:
        for v in all_versions(b):
            symmetric_board[v] = b

print({len(b) for b in boards})

print(len(symmetric_board.keys()))
print(len(set(symmetric_board.values())))


{9}
6046
850


In [136]:
print(
    'X-XOOXOXO' in boards,
    'X-XOOXOXO' in symmetric_board)

False False


It will also be useful to have a function for checking whether a board is winning for a player:

In [74]:
def winstate(state: str, player: str):
    wins = {"012","036","058","147","258","256","345","678"}
    for win in wins:
        for i in win:
            if state[int(i)] != player:
                break
        else: return True

In [139]:
from collections import deque


def initialise_game(beads = 10):      #-> MENACE:
    # d: MENACE = {}
    d= {}
    q: deque[str] = deque()
    q.append("---------")
    while q:
        s: str = q.popleft()
        if not sum(v in d for v in all_versions(s)):
            moves = [s[:i] + "O" + s[i+1:] for i in range(9) if s[i] == "-"] 
            unique_moves = []
            for m in moves:
                if not [mv for mv in all_versions(m) if mv in unique_moves]: unique_moves.append(m)
            d[s] = Box(s, unique_moves, [beads] * len(unique_moves))
            for m in unique_moves:
                if winstate(m, "O"): continue
                if not sum(v in d for v in all_versions(m)):
                    q.extend((m[:i] + "X" + m[i+1:] for i in range(9) if m[i] == "-"))
    return d

def init_Menace(beads = 10):
    d= {}
    q: deque[str] = deque()
    qset = set()
    q.append("---------")
    qset.add("---------")

    while q:
        s: str = q.popleft()
        if winstate(s,"X"): continue
        qset.remove(s)
        moves = list({symmetric_board[s[:i] + "O" + s[i+1:]] for i in range(9) if s[i] == "-"})
        d[s] = Box(s, moves, [beads for _ in moves])
        for m in moves:
            if winstate(m, "O"): continue
            for nexts in set(symmetric_board[m[:i] + "X" + m[i+1:]] for i in range(9) if m[i] == "-"):
                if nexts not in d and nexts not in qset:
                    q.append(nexts)
                    qset.add(nexts)
    return d
                
        
    
print(len(initialise_game(10)))
print(len(init_Menace(10)))

396
358


In [156]:
from random import choice, choices

def playturn(menace, state: str) -> str:
    for v in all_versions(state):
        if v in menace:
            state = v
            break
    else: 
        raise(ValueError("Please check game state is valid", 
                         all_versions(state), 
                         menace.keys()))
    
    box = menace[state]
    if sum(box.beads) <= 0: 
        return choice(box.moves)
    return choices(population = box.moves, weights = box.beads)[0]
    
    
@dataclass
class GameResult:
    win: str
    states: list[str]

def randomturn(s:str) -> str:
    return symmetric_board[choice([s[:i] + "X" + s[i+1:] for i in range(9) if s[i] == "-"])]
    
    
def randomgame(menace) -> GameResult:
    state = "---------"
    states = [state]
    while 1:
        if "-" not in state: return GameResult("DRAW", states)
        state = playturn(menace, state)
        states.append(state)
        if winstate(state, "O"): return GameResult("WIN", states)
        if "-" not in state: return GameResult("DRAW", states)
        state = randomturn(state)
        states.append(state)
        if winstate(state, "X"): return GameResult("LOSS", states)
        
    raise(Exception("This is just to appease Pylance!!!"))

In [141]:
def game_rep(game_state):
    if len(game_state) != 9:
        raise ValueError("Invalid game state. The string must be 9 characters long.")

    board = "|{}|{}|{}|\n|---|---|---|\n|{}|{}|{}|\n|{}|{}|{}|\n".format(
        game_state[0] if game_state[0] != "-" else " ",
        game_state[1] if game_state[1] != "-" else " ",
        game_state[2] if game_state[2] != "-" else " ",
        game_state[3] if game_state[3] != "-" else " ",
        game_state[4] if game_state[4] != "-" else " ",
        game_state[5] if game_state[5] != "-" else " ",
        game_state[6] if game_state[6] != "-" else " ",
        game_state[7] if game_state[7] != "-" else " ",
        game_state[8] if game_state[8] != "-" else " "
    )

    return board

In [154]:
def feedback(menace, result:GameResult, reward = 3, draw = 1, punishment = -1):
    if result.win == "Win":
        r = reward
    elif result.win == "LOSS":
        r = punishment
    else:
        r = draw
    states = result.states
    for i in range(0, len(states) -1, 2):
        
        box = menace[states[i]]
        idx = box.moves.index(states[i+1])
        box.beads[idx] = max(0, box.beads[idx] + r)

In [164]:
menace = init_Menace(10)

for _ in range(200):
    wins = 0
    draws = 0
    for _ in range(1000):
        game = randomgame(menace)
        feedback(menace, game, 3, 1, -1)
        wins += game.win in ("WIN")
        draws += game.win in ("DRAW")
    losses = 1000 - wins - draws
    print(f"W/D/L     {wins: 3}    {draws: 3}     {losses: 3}")

W/D/L      548     249      203
W/D/L      560     250      190
W/D/L      591     231      178
W/D/L      613     237      150
W/D/L      616     240      144
W/D/L      637     230      133
W/D/L      638     231      131
W/D/L      630     262      108
W/D/L      693     221      86
W/D/L      665     242      93
W/D/L      654     257      89
W/D/L      674     240      86
W/D/L      653     252      95
W/D/L      666     259      75
W/D/L      682     231      87
W/D/L      684     231      85
W/D/L      691     242      67
W/D/L      688     235      77
W/D/L      698     247      55
W/D/L      709     227      64
W/D/L      719     225      56
W/D/L      721     223      56
W/D/L      701     241      58
W/D/L      696     245      59
W/D/L      714     248      38
W/D/L      721     224      55
W/D/L      702     240      58
W/D/L      723     232      45
W/D/L      706     252      42
W/D/L      687     270      43
W/D/L      698     260      42
W/D/L      699     252      49


Box(state='---------', moves=['----O----', '---O-----', '--O------'], beads=[1578, 2658, 2868])
Box(state='----O--X-', moves=['--O-OX---', '-O--O--X-', '-X--O-O--', '-X-OO----'], beads=[247, 115, 212, 298])
Box(state='X---O----', moves=['-OX-O----', 'X---O-O--', '--O-O-X--', '--X-O--O-'], beads=[255, 203, 224, 94])
Box(state='----X--O-', moves=['O--OX----', '---OXO---', '-O--X-O--', '---OX--O-'], beads=[46, 125, 59, 173])
Box(state='-----O-X-', moves=['-X-OO----', '---X-O-O-', 'O----X-O-', '-X---O--O', '-O---X-O-', '-----OOX-', '---X--OO-'], beads=[150, 49, 56, 47, 148, 83, 147])
Box(state='-O-----X-', moves=['O--O-X---', '---X-O-O-', '-O-----XO', '-O--O--X-'], beads=[152, 55, 112, 64])
Box(state='-----O--X', moves=['O-X--O---', '-OX--O---', '-----OXO-', '-OX----O-', '--O--O--X', '-OX-O----', '--X--OO--'], beads=[201, 193, 91, 180, 27, 60, 26])
Box(state='-----OX--', moves=['--X-O--O-', '--X---OO-', '-----OXO-', '-O-O----X', '-OX----O-', '--X----OO', '-O----O-X'], beads=[114, 69, 93, 2